In [1]:
import numpy as np

In [54]:
arr = np.arange(16).reshape((4,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

---

* `arr.reshape((4,4))` aynı veriyi paylaşan yeni bir NumPy dizisi (view) oluşturur, ama bu farklı bir Python nesnesidir. Yani bellek blokları paylaşılsa da, obje kimlikleri farklıdır.

In [55]:
flat1 = arr.reshape(-1)
flat1

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

Not: `is` operatörü iki değişkenin tam olarak aynı Python nesnesine işaret edip etmediğini kontrol eder.

In [92]:
print(flat1.base is arr) #False --> flat1 ile arr aynı veriye bakıyor. flat1 veriyi arr'ın belleğinden okuyor.
print(flat1 is arr) #False

False
False


* `flat1.base` ifadesi **flat1** in veriyi başka hangi nesneden aldığı bilgisini tutar. (`arr` değil. `np.arange(16)`nın kendisi)
* `flat1.base is arr` ifadesi **True** döndürmez. `flat1 is arr` ifadesi hiç döndürmez. Ama `flat1.base is np.arange(16)` nın sonucu **True** olur.

* Bu `np.shares_memory()`ile de kontrol edilebilir. 

In [57]:
print(np.shares_memory(arr, flat1))

True


* Fakat ne garip ki `shares_memory()` **True** döndürür. Neden?
* `shares_memory`verilen iki veri nesnesinin memory yoluna bakar eğer çakışıyorsa **True** döndürür. 
* Biraz görselleştirelim: 

`arr` | `np.arange(16) -> np.arange(16).reshape((4,4))` iken, 

`flat1` | `np.arange(16) -> np.arange(16).reshape((4,4)) -> np.reshape(-1)` dir. 

***Tüm bu işlemler *view* kapsamına girer.***

---

In [64]:
flat2 = arr.ravel()
flat2

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [61]:
flat2.base is arr

False

* `ravel()` de `reshape()` gibi aynı veriye farklı obje ile işaret ediyor.

In [62]:
print(np.shares_memory(arr, flat2))

True


****özet****

In [70]:
import numpy as np

# 1) Orijinal veri bloğunu yaratan dizi
base0 = np.arange(16)           # gerçek “owner”

# 2) base0’dan türetilmiş bir view
arr = base0.reshape((4,4))      # arr.base is base0 → True

# 3) arr’dan reshape ile tek boyuta indirdin
flat1 = arr.reshape(-1)
# flat1.base is base0 → True 
# flat1.base is arr   → False  (çünkü arr zaten base0’dan gelen bir view idi)

# 4) arr’dan ravel ile tek boyuta indirdin
flat2 = arr.ravel()
# flat2.base is base0 → True
# flat2.base is arr   → False

---

In [66]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [67]:
flat3 = arr.flatten()
flat3

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [78]:
print(flat3.base)           # -> None olmalı
print(flat3.base is arr)    # -> False olmalı
print(np.shares_memory(arr, flat3))  # -> False

None
False
False


---

* `base0 = np.arange(16)`

* `arr` **base0** ın `reshape((4,4))` uygulanmış hali

* `view1` **base0** ın `view()` uygulanmış hali

* `view1` kendi içinde bir kaç **shape** değişikliğine uğruyor.

* `view2` **view1** ın `reshape((4,4))` uygulanmış hali

In [129]:
base0 = np.arange(16)
arr = base0.reshape((4,4))
print(arr.base is base0)

True


In [102]:
view1 = base0.view()
view1

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [121]:
print(view1.base is arr) #false
print(arr.base is view1) #false

False
False


In [95]:
np.shares_memory(arr, view1) #ortak bir yerde memoryler kesişiyor (np.arange(16))

True

In [97]:
print(view1.base is arr)
print(view1 is arr)
print(view1.base is base0)
print(view1 is base0)

False
False
True
False


In [ ]:
view1.shape = (2,8) #(16,) -> (2,8)
view1

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15]])

In [ ]:
print(view1.base is base0) # True -> base0'dan türetilmiş

True


In [ ]:
view1.shape = (16,) # (2,8) -> (16,)
view1

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [ ]:
print(view1.base is base0) #True -> eski haline döndürdüm.

True


In [110]:
view2 = view1.reshape((4,4))

In [122]:
print(view2.base is view1) # false -> doğrusu: base0
print(np.shares_memory(view2, view1)) # true -> farklı nesne olsalarda aynı memory den okurlar.

False
True


---

In [136]:
base0 = np.arange(16)
arr = base0.reshape((4,4))
print(arr.base is base0)

True


In [137]:
tmp = arr.reshape(2,8).reshape(8,2).reshape(4,4)
tmp

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

* Burada her **reshape** bir **view** yaratıyor, ama `.base` en dipteki esas veri sahibi olan **base0’ı** gösterir.

In [140]:
print(arr.base is base0) #true
print(tmp.base is base0) #true
print(tmp.base is arr.base) #true

True
True
True


---

****`flat1,flat2,flat3,view1,tmp` değişkenlerin tuttuğu arrayler üzerinde değişiklirler yapacağız. hangisinin değişikliği `arr`'a yansıyacak göreceğiz.****

* **base0** *=* **np.arange(16)**

* **arr** *=* **base0.reshape((4,4))**

In [160]:
base0 = np.arange(16)
arr = base0.reshape((4,4))
flat1 = arr.reshape(-1)
print(arr)
flat1[0] = 100
print(arr)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
[[100   1   2   3]
 [  4   5   6   7]
 [  8   9  10  11]
 [ 12  13  14  15]]


In [161]:
base0 = np.arange(16)
arr = base0.reshape((4,4))
flat2 = arr.ravel()
print(arr)
flat2[0] = 100
print(arr)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
[[100   1   2   3]
 [  4   5   6   7]
 [  8   9  10  11]
 [ 12  13  14  15]]


In [163]:
base0 = np.arange(16)
arr = base0.reshape((4,4))
flat3 = arr.flatten()
print(arr)
flat3[0] = 100
print(arr)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


* ****`flatten()` **view** değil **copy** döner.****

In [159]:
base0 = np.arange(16)
arr = base0.reshape((4,4))
tmp = arr.reshape(2,8).reshape(8,2).reshape(4,4)
print(arr)
tmp[0][0] = 100
print(arr)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
[[100   1   2   3]
 [  4   5   6   7]
 [  8   9  10  11]
 [ 12  13  14  15]]


In [164]:
base0 = np.arange(16)
arr = base0.reshape((4,4))
view1 = base0.view()
print(arr)
view1[0] = 100
print(arr)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
[[100   1   2   3]
 [  4   5   6   7]
 [  8   9  10  11]
 [ 12  13  14  15]]
